<a href="https://colab.research.google.com/github/Pointbreak82/Toxic-Comment-2/blob/main/Toxic_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import timeit


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Get the data

full_data_path = "/content/drive/MyDrive/Toxic/train.csv"
test_data_X_path = "/content/drive/MyDrive/Toxic/test.csv"
test_data_y_path = "/content/drive/MyDrive/Toxic/test_labels.csv"

full_data = pd.read_csv(full_data_path)
test_data_X = pd.read_csv(test_data_X_path)
test_data_y = pd.read_csv(test_data_y_path)

In [4]:
full_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
print(f"There are {len(full_data)} observations in full data.")

There are 159571 observations in full data.


In [6]:
test_data_X.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
test_data_y.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [8]:
# Merging the X and y part together
test_dataframe = pd.merge(test_data_X, test_data_y, how="inner", on="id")

# Remove all the rows having missing values (-1)
test_dataframe = test_dataframe[test_dataframe["toxic"] != -1].reset_index(drop=True)

test_dataframe.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
34758,8acd629d04ba51d5,""" \n\n Please do not vandalize pages, as you ...",0,0,0,0,0,0
28424,7144f2c4d749ce00,Mistake \n You made a mistake while editing un...,0,0,0,0,0,0
35456,8da3eba117bae0ba,HELLO HOW ARE YOU I AM FINE THANKS FOR ASKING ...,0,0,0,0,0,0
29928,77548ab8f7edfdc1,"== Sorry for annoying,k == \n\n I'M So Sorry t...",0,0,0,0,0,0
19031,4b7bb37d414b7532,""" \n\n ==YAY For Dino's!!== \n\n \n {| class=...",0,0,0,0,0,0


In [9]:
print(f"We have {len(test_dataframe)} observations in test data.")

We have 63978 observations in test data.


In [10]:
# Remove the unnecessary data

del test_data_X
del test_data_y

In [11]:
full_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [12]:
full_data["comment_text"].values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [13]:
full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [14]:
# Convert the dataframes into tensorflow Dataset objects

train_data = tf.data.Dataset.from_tensor_slices(
    (
        full_data["comment_text"].tolist(),
        full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

test_data = tf.data.Dataset.from_tensor_slices(
    (
        test_dataframe["comment_text"].tolist(),
        test_dataframe[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

In [15]:
batch_size = 16

train_data = train_data.cache().shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_data = test_data.cache().batch(batch_size)

In [16]:
max_tokens = 100000

start = timeit.default_timer()

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize='lower_and_strip_punctuation',
    output_mode="int",
    output_sequence_length=1800
)

text_vectorization.adapt(full_data["comment_text"].tolist())

end = timeit.default_timer()

print(f"It took {end - start} seconds to adapt.")

It took 26.439872414000035 seconds to adapt.


In [17]:
# Remove the unnecessary data

del full_data
del test_dataframe

In [18]:
model = Sequential([
    text_vectorization,
    layers.Embedding(max_tokens+1, 32),
    layers.Bidirectional(layers.LSTM(32, return_sequences=False)),
    layers.Dense(256, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(6, activation="sigmoid")
])

In [19]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=metrics.BinaryAccuracy()
)

In [20]:
train_split = 0.8

train_data = train_data.take(int(len(train_data) * train_split))
valid_data = train_data.skip(int(len(train_data) * train_split))

In [21]:
print("The cardinality of train data is ", train_data.cardinality().numpy())
print("The cardinality of validation data is ", valid_data.cardinality().numpy())
print("The cardinality of test data is ", test_data.cardinality().numpy())

The cardinality of train data is  7979
The cardinality of validation data is  1596
The cardinality of test data is  3999


In [22]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        start_from_epoch=1,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-6
    )
]

In [23]:
start = timeit.default_timer()

model_history = model.fit(
    train_data,
    epochs=15,
    verbose=1,
    callbacks=callbacks,
    validation_data=valid_data
).history

end = timeit.default_timer()

print(f"It took {end - start} seconds to train the model.")

Epoch 1/15
7979/7979 [==============================] - 954s 119ms/step - loss: 0.0604 - binary_accuracy: 0.9795 - val_loss: 0.0482 - val_binary_accuracy: 0.9826 - lr: 0.0010
Epoch 2/15
7979/7979 [==============================] - 838s 105ms/step - loss: 0.0444 - binary_accuracy: 0.9831 - val_loss: 0.0362 - val_binary_accuracy: 0.9859 - lr: 0.0010
Epoch 3/15
7979/7979 [==============================] - 827s 104ms/step - loss: 0.0380 - binary_accuracy: 0.9852 - val_loss: 0.0324 - val_binary_accuracy: 0.9881 - lr: 0.0010
Epoch 4/15
7979/7979 [==============================] - 838s 105ms/step - loss: 0.0325 - binary_accuracy: 0.9871 - val_loss: 0.0305 - val_binary_accuracy: 0.9892 - lr: 0.0010
Epoch 5/15
7979/7979 [==============================] - 805s 101ms/step - loss: 0.0282 - binary_accuracy: 0.9890 - val_loss: 0.0218 - val_binary_accuracy: 0.9922 - lr: 0.0010
Epoch 6/15
7979/7979 [==============================] - 807s 101ms/step - loss: 0.0244 - binary_accuracy: 0.9907 - val_loss: 

In [24]:
model.evaluate(test_data)

3999/3999 [==============================] - 166s 41ms/step - loss: 0.1837 - binary_accuracy: 0.9488


[0.18373829126358032, 0.9487975239753723]

In [25]:
text = "Yo bitch he is more succesful then you'll be in your life"
predictions = model.predict(np.array([text])) > 0.5
prediction_text = ""
for i, col in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    prediction_text += f"{col:<{20}}: {predictions[0][i]:<{5}}\n"
print(prediction_text)

1/1 [==============================] - 1s 780ms/step
toxic               : 1    
severe_toxic        : 1    
obscene             : 1    
threat              : 0    
insult              : 1    
identity_hate       : 0    

